In [ ]:
# 기본 라이브러리 설치
!pip install torch
!pip install transformers
!pip install sentencepiece

# KoBERT 전용 토크나이저 설치
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-hf2sd6sx/kobert-tokenizer_2e38c098d37a4235b8c5614cbdd6adb9
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-hf2sd6sx/kobert-tokenizer_2e38c098d37a4235b8c5614cbdd6adb9
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 5c46b1c68e4755b54879431bd302db621f4d2f47
  Preparing metadata (setup.py) ... done


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **학습 시간 단축**

In [ ]:
# 리뷰 판별 모델 다운로드 및 테스트 코드

# 라이브러리 불러오기
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import os

# Google Drive 연동
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# 파일 경로 수정
try:
    df = pd.read_csv('/content/drive/MyDrive/result_11_13.csv', encoding='utf-8')
except UnicodeDecodeError:
    df = pd.read_csv('/content/drive/MyDrive/result_11_13.csv', encoding='utf-8-sig')

# KoBERT 모델과 토크나이저 불러오기
model_name = 'monologg/kobert'
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# GPU 설정
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 드롭아웃과 추가적인 레이어를 적용한 커스텀 모델
class CustomKoBERTModel(nn.Module):
    def __init__(self, model_name, num_labels=2, dropout_prob=0.3):  # 드롭아웃을 0.3으로 조정
        super(CustomKoBERTModel, self).__init__()
        self.bert = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels, trust_remote_code=True)
        self.dropout = nn.Dropout(dropout_prob)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)
        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        logits = self.dropout(outputs.logits)

        if labels is not None:
            loss = self.loss_fn(logits, labels)
            return loss, logits
        else:
            return logits

# 매번 학습 시작 시 모델을 새로 초기화하여 과적합 방지
model = CustomKoBERTModel(model_name).to(device)

# 현재 경로 설정 (저장할 위치로 변경 가능)
path = '/content/drive/MyDrive/models/'

# 필요한 경우 가중치 저장
if not os.path.exists(path):
    os.makedirs(path)

# 데이터셋 클래스 정의 (수정된 부분)
class ReviewDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        # 'Positive'와 'Negative'를 숫자로 변환합니다.
        self.labels = [1 if label == 'Positive' else 0 for label in labels]
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# 데이터로더 생성
train_dataset = ReviewDataset(df['상품평'].tolist(), df['sentiment'].tolist(), tokenizer)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# 학습 설정
optimizer = optim.AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)  # 학습률을 증가시킴
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

# 간단한 학습 루프 (에포크 수 증가)
model.train()
for epoch in range(5):  # 에포크 수 5로 증가
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        loss, _ = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    scheduler.step()  # 스케줄러 업데이트
    print(f'Epoch {epoch + 1}, Loss: {total_loss / len(train_loader):.4f}')

# 학습 후 리뷰 판별 결과 출력 함수 (긍정, 부정 확률 출력)
def print_review_predictions(reviews, model, tokenizer, device):
    print("\n리뷰 판별 결과:")
    model.eval()
    for review in reviews:
        with torch.no_grad():
            inputs = tokenizer.encode_plus(
                review,
                add_special_tokens=True,
                max_length=128,
                padding='max_length',
                truncation=True,
                return_tensors='pt'
            )
            input_ids = inputs['input_ids'].to(device)
            attention_mask = inputs['attention_mask'].to(device)

            logits = model(input_ids=input_ids, attention_mask=attention_mask)
            probabilities = F.softmax(logits, dim=1).squeeze()  # 소프트맥스 적용하여 확률 계산
            positive_prob = probabilities[1].item() * 100
            negative_prob = probabilities[0].item() * 100

            result = "긍정" if positive_prob > negative_prob else "부정"
            print(f"리뷰: '{review}' \n 판별 결과: {result} \n (긍정: {positive_prob:.2f}%, 부정: {negative_prob:.2f}%)")


Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/263 [00:00<?, ?B/s]

tokenization_kobert.py:   0%|          | 0.00/10.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/monologg/kobert:
- tokenization_kobert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

vocab.txt:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Loss: 0.0838
Epoch 2, Loss: 0.0691
Epoch 3, Loss: 0.0650
Epoch 4, Loss: 0.0665
Epoch 5, Loss: 0.0646


# **Ko 선생님**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch.nn.functional as F
from tqdm import tqdm

class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        encoding = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

class SentimentAnalyzer:
    def __init__(self, model_name='monologg/kobert'):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        self.model = AutoModelForSequenceClassification.from_pretrained(
            model_name,
            num_labels=2,
            trust_remote_code=True
        ).to(self.device)

    def prepare_data(self, df, text_column='상품평', label_column='sentiment', test_size=0.2):
        # Convert labels to numeric
        label_map = {'Positive': 1, 'Negative': 0, 'Neutral': 0}  # Treating Neutral as Negative
        df['label_numeric'] = df[label_column].map(label_map)

        # Split data
        train_texts, val_texts, train_labels, val_labels = train_test_split(
            df[text_column].values,
            df['label_numeric'].values,
            test_size=test_size,
            random_state=42
        )

        # Create datasets
        train_dataset = SentimentDataset(train_texts, train_labels, self.tokenizer)
        val_dataset = SentimentDataset(val_texts, val_labels, self.tokenizer)

        return train_dataset, val_dataset

    def train(self, train_dataset, val_dataset, epochs=5, batch_size=16, learning_rate=2e-5):
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size)

        optimizer = torch.optim.AdamW(self.model.parameters(), lr=learning_rate)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

        best_accuracy = 0

        for epoch in range(epochs):
            # Training
            self.model.train()
            total_loss = 0
            correct_predictions = 0
            total_predictions = 0

            progress_bar = tqdm(train_loader, desc=f'Epoch {epoch + 1}/{epochs} [Train]')
            for batch in progress_bar:
                optimizer.zero_grad()

                input_ids = batch['input_ids'].to(self.device)
                attention_mask = batch['attention_mask'].to(self.device)
                labels = batch['labels'].to(self.device)

                outputs = self.model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    labels=labels
                )

                loss = outputs.loss
                logits = outputs.logits

                loss.backward()
                optimizer.step()

                total_loss += loss.item()

                _, predicted = torch.max(logits, 1)
                correct_predictions += (predicted == labels).sum().item()
                total_predictions += labels.size(0)

                progress_bar.set_postfix({
                    'loss': f'{total_loss/(progress_bar.n+1):.4f}',
                    'accuracy': f'{correct_predictions/total_predictions*100:.2f}%'
                })

            # Validation
            self.model.eval()
            val_loss = 0
            val_correct = 0
            val_total = 0

            with torch.no_grad():
                for batch in tqdm(val_loader, desc='Validation'):
                    input_ids = batch['input_ids'].to(self.device)
                    attention_mask = batch['attention_mask'].to(self.device)
                    labels = batch['labels'].to(self.device)

                    outputs = self.model(
                        input_ids=input_ids,
                        attention_mask=attention_mask,
                        labels=labels
                    )

                    val_loss += outputs.loss.item()
                    logits = outputs.logits

                    _, predicted = torch.max(logits, 1)
                    val_correct += (predicted == labels).sum().item()
                    val_total += labels.size(0)

            val_accuracy = val_correct / val_total * 100
            print(f'\nValidation Accuracy: {val_accuracy:.2f}%')

            if val_accuracy > best_accuracy:
                best_accuracy = val_accuracy
                # Save the best model
                torch.save(self.model.state_dict(), 'best_sentiment_model.pth')

            scheduler.step()

    def predict(self, texts, batch_size=16):
        self.model.eval()
        predictions = []
        probabilities = []

        dataset = SentimentDataset(texts, [0] * len(texts), self.tokenizer)  # Dummy labels
        dataloader = DataLoader(dataset, batch_size=batch_size)

        with torch.no_grad():
            for batch in dataloader:
                input_ids = batch['input_ids'].to(self.device)
                attention_mask = batch['attention_mask'].to(self.device)

                outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
                logits = outputs.logits

                probs = F.softmax(logits, dim=1)
                _, preds = torch.max(logits, 1)

                predictions.extend(preds.cpu().numpy())
                probabilities.extend(probs.cpu().numpy())

        return predictions, probabilities

# 사용 예시
def main():
    # 데이터 로드
    try:
        df = pd.read_csv('/content/drive/MyDrive/result_today.csv', encoding='utf-8')
    except UnicodeDecodeError:
        df = pd.read_csv('/content/drive/MyDrive/result_today.csv', encoding='utf-8-sig')

    # 분석기 초기화
    analyzer = SentimentAnalyzer()

    # 데이터 준비
    train_dataset, val_dataset = analyzer.prepare_data(df)

    # 모델 학습
    analyzer.train(train_dataset, val_dataset, epochs=5)

    # 예측 테스트
    test_texts = [
        "맛있어요 또 구매할게요",
        "배송이 너무 늦고 상품도 별로에요"
    ]
    predictions, probabilities = analyzer.predict(test_texts)

    # 결과 출력
    for text, pred, prob in zip(test_texts, predictions, probabilities):
        sentiment = "긍정" if pred == 1 else "부정"
        print(f"\n리뷰: {text}")
        print(f"판별 결과: {sentiment}")
        print(f"긍정 확률: {prob[1]*100:.2f}%, 부정 확률: {prob[0]*100:.2f}%")

if __name__ == "__main__":
    main()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Validation: 100%|██████████| 2369/2369 [01:12<00:00, 32.65it/s]



Validation Accuracy: 99.53%


Validation: 100%|██████████| 2369/2369 [01:12<00:00, 32.70it/s]



Validation Accuracy: 99.59%


Validation: 100%|██████████| 2369/2369 [01:12<00:00, 32.68it/s]



Validation Accuracy: 99.76%


Validation: 100%|██████████| 2369/2369 [01:12<00:00, 32.68it/s]



Validation Accuracy: 99.87%


Validation: 100%|██████████| 2369/2369 [01:12<00:00, 32.67it/s]


Validation Accuracy: 99.83%

리뷰: 맛있어요 또 구매할게요
판별 결과: 긍정
긍정 확률: 100.00%, 부정 확률: 0.00%

리뷰: 배송이 너무 늦고 상품도 별로에요
판별 결과: 부정
긍정 확률: 0.00%, 부정 확률: 100.00%


다기 저장

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        encoding = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

class SentimentAnalyzerLoader:
    def __init__(self, model_name='monologg/kobert'):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        self.model = AutoModelForSequenceClassification.from_pretrained(
            model_name,
            num_labels=2,
            trust_remote_code=True
        ).to(self.device)

    def load_model(self, model_path='/content/drive/MyDrive/models/best_sentiment_model.pth'):
        """
        학습된 모델을 로드하는 메서드
        """
        checkpoint = torch.load(model_path, map_location=self.device)

        # model_state_dict가 존재하는지 확인
        if 'model_state_dict' in checkpoint:
            self.model.load_state_dict(checkpoint['model_state_dict'])
            print(f"모델이 성공적으로 로드되었습니다. Epoch: {checkpoint.get('epoch')}, 정확도: {checkpoint.get('accuracy'):.2f}%")
        else:
            # 만약 model_state_dict 키가 없다면 state_dict만 포함된 것으로 가정
            self.model.load_state_dict(checkpoint)
            print("모델 state_dict만 포함된 파일을 성공적으로 로드하였습니다.")

        self.model.eval()

    def save_model(self, epoch, accuracy, optimizer, model_path='/content/drive/MyDrive/models/best_sentiment_model.pth'):
        """
        학습된 모델을 구글 드라이브에 저장하는 메서드
        """
        torch.save({
            'epoch': epoch,
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'accuracy': accuracy,
        }, model_path)
        print(f"모델이 {model_path}에 성공적으로 저장되었습니다.")

    def predict(self, texts, batch_size=16):
        """
        로드한 모델로 텍스트 데이터를 예측하는 메서드
        """
        self.model.eval()
        predictions = []
        probabilities = []

        dataset = SentimentDataset(texts, [0] * len(texts), self.tokenizer)  # Dummy labels
        dataloader = DataLoader(dataset, batch_size=batch_size)

        with torch.no_grad():
            for batch in dataloader:
                input_ids = batch['input_ids'].to(self.device)
                attention_mask = batch['attention_mask'].to(self.device)

                outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
                logits = outputs.logits

                probs = F.softmax(logits, dim=1)
                _, preds = torch.max(logits, 1)

                predictions.extend(preds.cpu().numpy())
                probabilities.extend(probs.cpu().numpy())

        return predictions, probabilities


# **모델 저장**

In [ ]:
# Google Drive 연동
from google.colab import drive
drive.mount('/content/drive')

# 현재 경로 설정 (저장할 위치로 변경 가능)
path = '/content/drive/MyDrive/models/'

# torch 모듈 import
import torch

# 모델 가중치만 저장 (모델 객체가 'model'이라는 이름으로 정의되어 있다고 가정)
torch.save(model.state_dict(), path + '진짜 진짜 최종 모델.pt')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


진짜 테스트


In [ ]:
if __name__ == "__main__":
    # 모델을 로드하고 테스트하는 코드
    analyzer = SentimentAnalyzerLoader()
    analyzer.load_model('/content/drive/MyDrive/첫번째 리뷰데이터.pt')

    # 테스트할 텍스트 예시
    test_texts = [
        '이 반찬 정말 맛있어요!',
        '음식이 너무 짜고 별로였어요.',
        '다음에도 또 주문하고 싶습니다.',
        '배송이 늦어서 실망했습니다.',
 ]

    # 예측 수행
    predictions, probabilities = analyzer.predict(test_texts)

    # 결과 출력
    for text, pred, prob in zip(test_texts, predictions, probabilities):
        sentiment = "긍정" if pred == 1 else "부정"
        print(f"\n리뷰: {text}")
        print(f"판별 결과: {sentiment}")
        print(f"긍정 확률: {prob[1]*100:.2f}%, 부정 확률: {prob[0]*100:.2f}%")

NameError: name 'SentimentAnalyzerLoader' is not defined

# **테스트**

In [ ]:
# 리뷰 판별 모델 다운로드 및 테스트 코드

# 라이브러리 불러오기
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig
import torch.nn as nn
import torch.nn.functional as F

# GPU 설정
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# KoBERT 모델과 토크나이저 불러오기
model_name = 'monologg/kobert'
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
config = AutoConfig.from_pretrained(model_name)

# 학습된 모델 가중치만 불러오기
model_path = '/content/drive/MyDrive/models/진짜 진짜 최종 모델.pt'
try:
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, trust_remote_code=True)
    model.load_state_dict(torch.load(model_path, map_location=device), strict=False)
    model.to(device)
    model.eval()
except FileNotFoundError:
    print(f"Error: Model file not found at {model_path}. Please check the path and try again.")
    model = None

# 학습 후 리뷰 판별 테스트용 리뷰 텍스트 (이미지에서 추출된 리뷰들)
test_reviews = [
        '이 반찬 정말 맛있어요!',
        '음식이 너무 짜고 별로였어요.',
        '다음에도 또 주문하고 싶습니다.',
        '배송이 늦어서 실망했습니다.',]

# 리뷰 판별 결과 출력 함수 (긍정, 부정 확률 출력)
def print_review_predictions(reviews, model, tokenizer, device):
    print("\n리뷰 판별 결과:")
    model.eval()
    for review in reviews:
        with torch.no_grad():
            inputs = tokenizer.encode_plus(
                review,
                add_special_tokens=True,
                max_length=128,
                padding='max_length',
                truncation=True,
                return_tensors='pt'
            )
            input_ids = inputs['input_ids'].to(device)
            attention_mask = inputs['attention_mask'].to(device)

            logits = model(input_ids=input_ids, attention_mask=attention_mask).logits
            probabilities = F.softmax(logits, dim=1)[0]  # 소프트맥스 적용하여 확률 계산
            positive_prob = probabilities[1].item() * 100
            negative_prob = probabilities[0].item() * 100

            result = "긍정" if positive_prob > negative_prob else "부정"
            print(f"리뷰: '{review}' \n 판별 결과: {result} \n (긍정: {positive_prob:.2f}%, 부정: {negative_prob:.2f}%)")

# 학습 후 리뷰 판별 결과 출력
if model is not None:
    print_review_predictions(test_reviews, model, tokenizer, device)
else:
    print("모델을 불러오지 못했습니다. 파일 경로를 확인해주세요.")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-10-d242c248a4df>:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recom


리뷰 판별 결과:
리뷰: '이 반찬 정말 맛있어요!' 
 판별 결과: 긍정 
 (긍정: 53.88%, 부정: 46.12%)
리뷰: '음식이 너무 짜고 별로였어요.' 
 판별 결과: 부정 
 (긍정: 40.63%, 부정: 59.37%)
리뷰: '다음에도 또 주문하고 싶습니다.' 
 판별 결과: 부정 
 (긍정: 42.53%, 부정: 57.47%)
리뷰: '배송이 늦어서 실망했습니다.' 
 판별 결과: 긍정 
 (긍정: 52.46%, 부정: 47.54%)
